In [22]:
import os
from groq import Groq
import dotenv
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
import json

In [23]:
dotenv.load_dotenv('.env') 
groq_api_key = os.getenv("GROQ_API_KEY")
client = Groq(api_key=groq_api_key)
MODEL = 'llama-3.3-70b-versatile'

In [24]:
# Khởi tạo vector database với FAISS
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={'device': 'cpu'}
)

In [25]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "rag_aio",
            "description": "Trả lời câu hỏi dựa trên dữ liệu từ tài liệu AIO PDF (ví dụ: AI Agent là gì)",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "Câu hỏi của người dùng cần trả lời"
                    },
                },
                "required": ["question"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "rag_billionares",
            "description": "Trả lời câu hỏi dựa trên dữ liệu từ tài liệu Billionares PDF (ví dụ: Có bao nhiêu tỷ phú ở Mỹ)",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "Câu hỏi của người dùng về tỷ phú"
                    }
                },
                "required": ["question"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "rag_economic",
            "description": "Trả lời câu hỏi dựa trên dữ liệu từ tài liệu Economic PDF (về kinh tế và tài chính)",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "Câu hỏi của người dùng về kinh tế"
                    }
                },
                "required": ["question"]
            }
        }
    }
]

In [26]:
from sentence_transformers import CrossEncoder


# rerank search results
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

def retrieve_and_re_rank_advanced(vector_db, query, k=10):
    # Lấy kết quả từ Vector Database
    docs_with_scores = vector_db.similarity_search_with_score(query, k=k)
    
    # Chuẩn bị dữ liệu cho Cross-Encoder
    doc_texts = [doc.page_content for doc, _ in docs_with_scores]
    pairs = [[query, doc] for doc in doc_texts]
    
    # Dùng Cross-Encoder để đánh giá lại
    rerank_scores = cross_encoder.predict(pairs)
    
    # Sắp xếp lại theo score mới
    ranked_docs = sorted(zip(doc_texts, rerank_scores), key=lambda x: x[1], reverse=True)
    
    results = [doc for doc, _ in ranked_docs]
    scores = [score for _, score in ranked_docs]
    
    return results, scores


In [ ]:
def rag_aio(question: str):
    vector_db = FAISS.load_local('/home/thangcn/Downloads/datn/faiss_db/pdf_aio', embeddings, allow_dangerous_deserialization=True)
    retrieved_docs, scores = retrieve_and_re_rank_advanced(vector_db, question)
        # Gửi yêu cầu đến mô hình Groq
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are a helpful assistant that utilizes retrieved information."},
            {"role": "user", "content": f"Context:\n{retrieved_docs[0]}\n\nQuestion: {question}"},
        ],
        model= MODEL,
    )
    print(chat_completion.choices[0].message.content)
    return chat_completion.choices[0].message.content

def rag_billionares(question: str):
    vector_db = FAISS.load_local('/home/thangcn/Downloads/datn/faiss_db/pdf_billionares', embeddings, allow_dangerous_deserialization=True)
    retrieved_docs, scores = retrieve_and_re_rank_advanced(vector_db, question)
        # Gửi yêu cầu đến mô hình Groq
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are a helpful assistant that utilizes retrieved information."},
            {"role": "user", "content": f"Context:\n{retrieved_docs[0]}\n\nQuestion: {question}"},
        ],
        model= MODEL,
    )
    print(chat_completion.choices[0].message.content)
    return chat_completion.choices[0].message.content

def rag_economic(question: str):
    vector_db = FAISS.load_local('/home/thangcn/Downloads/datn/faiss_db/pdf_economic', embeddings, allow_dangerous_deserialization=True)
    retrieved_docs, scores = retrieve_and_re_rank_advanced(vector_db, question)
        # Gửi yêu cầu đến mô hình Groq
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are a helpful assistant that utilizes retrieved information."},
            {"role": "user", "content": f"Context:\n{retrieved_docs[0]}\n\nQuestion: {question}"},
        ],
        model= MODEL,
    )
    print(chat_completion.choices[0].message.content)
    return chat_completion.choices[0].message.content

In [35]:
def run_conversation(user_prompt):
    messages = [
        {
            "role": "system",
            "content": "You are a function calling LLM that uses the data extracted from the functions to answer questions around mutual funds, UPI transactions, health insurance policies, and cash loans."
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]

    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_tokens=4096
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # ChatCompletionMessage(
    #     content=None, 
    #     role='assistant', 
    #     function_call=None, 
    #     reasoning=None, 
    #     tool_calls=[
    #         ChatCompletionMessageToolCall(
    #             id='call_zd18', 
    #             function=Function(arguments='{"transaction_id":"TX123"}', name='upi'), 
    #             type='function'
    #         )
    #     ]
    # )
    # print(response_message)
    # print('-' * 100)
    if tool_calls:
        # try:
        available_functions = {
            "rag_aio": rag_aio,
            "rag_billionares": rag_billionares,
            "rag_economic": rag_economic
        }

        messages.append(response_message)
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            print(function_name)
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(**function_args)
            
            return function_response
        #     messages.append({
        #         "tool_call_id": tool_call.id,
        #         "role": "tool",
        #         "name": function_name,
        #         "content": function_response,
        #     })
            
        # second_response = client.chat.completions.create(
        #     model=MODEL,
        #     messages=messages
        # )

        # final_response = second_response.choices[0].message.content
        # except Exception as e:
        #     return "Sorry I'm not able to answer that question"
    else:
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            max_tokens=4096
        )
        messages.append(response_message)
        final_response = response.choices[0].message.content

    return final_response

In [38]:
input = "AI Agent là gì?"
response = run_conversation(input)
print(response)

rag_aio
AI Agent (hay Agent) là một thành phần trong lĩnh vực Trí tuệ Nhân tạo (AI) cho phép chatbot thực hiện nhiều công việc hơn, không chỉ giới hạn trong việc trò chuyện và trả lời câu hỏi. Agent có thể kết nối với các nguồn dữ liệu, thực hiện các tác vụ phức tạp, tìm kiếm thông tin mới, thay đổi dữ liệu hiện có, và thực hiện các công việc tự động hóa khác.

Agent là một bước tiến quan trọng trong việc phát triển các hệ thống AI, vì nó cho phép chatbot trở thành một công cụ hữu ích hơn, linh hoạt hơn và có thể thực hiện nhiều nhiệm vụ khác nhau. Với Agent, chatbot có thể trở thành một trợ lý ảo thực sự, giúp người dùng thực hiện các công việc hàng ngày và cung cấp thông tin một cách hiệu quả hơn.
